In [1]:
# import
from keras.models import Model, load_model
from keras.backend import clear_session
import numpy as np
import gc

Constant

In [2]:
# constant
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), "..\\00_Constant"))
if sys.path.count(parent_dir) == 0:
    sys.path.append(parent_dir)
import fyp_constants as constants


DATA_FOLDER_PATH            = constants.DATA_FOLDER_PATH
DATA_PROCESSED_FOLDER_NM    = constants.DATA_PROCESSED_FOLDER_NM
MODEL_FOLDER_PATH           = constants.MODEL_FOLDER_PATH

DATAX_MEL64_NM      = constants.DATAX_MEL64_NM
DATAX_MEL128_NM     = constants.DATAX_MEL128_NM
DATAX_MEL256_NM     = constants.DATAX_MEL256_NM
DATAX_WAVE_NM       = constants.DATAX_WAVE_NM
DATAX_WAVE_HALF_NM  = constants.DATAX_WAVE_HALF_NM

DATAY_NO_PROC_NM            = constants.DATAY_NO_PROC_NM
DATAY_NO_PROC_HALF_NM       = constants.DATAY_NO_PROC_HALF_NM
DATAY_PROC_VIB_NM           = constants.DATAY_PROC_VIB_NM
DATAY_PROC_VIB_GEN_NM       = constants.DATAY_PROC_VIB_GEN_NM
DATAY_PROC_VIB_GEN_BRE_NM   = constants.DATAY_PROC_VIB_GEN_BRE_NM
DATAY_PROC_VIB_GEN_BRI_NM   = constants.DATAY_PROC_VIB_GEN_BRI_NM
DATAY_PROC_VIB_GEN_GWL_NM   = constants.DATAY_PROC_VIB_GEN_GWL_NM
DATAY_PROC_GEN_MINMAXNORM   = constants.DATAY_PROC_GEN_MINMAXNORM

XSHAPE_MEL64        = constants.XSHAPE_MEL64
XSHAPE_MEL128       = constants.XSHAPE_MEL128
XSHAPE_MEL256       = constants.XSHAPE_MEL256
XSHAPE_WAVE         = constants.XSHAPE_WAVE
YSHAPE              = constants.YSHAPE
SAMPLING_RATE       = constants.SAMPLING_RATE
PARAMETERTYPE_LS    = constants.PARAMETERTYPE_LS

Helper Function

In [4]:
def loadData(dataset:int, dataX_nm:str, dataY_nm:str="data_y"):
    PROCESSDATA_FOLDER_PATH = f"{DATA_FOLDER_PATH}\\{dataset}\\{DATA_PROCESSED_FOLDER_NM}"
    x = np.load(f"{PROCESSDATA_FOLDER_PATH}\\{dataX_nm}.npz")["data"]
    y = np.load(f"{PROCESSDATA_FOLDER_PATH}\\{dataY_nm}.npz")["data"]
    return (x,y)

Model Construction

In [ ]:
from keras.layers import Input
from keras.layers import BatchNormalization, Flatten, Dense, Dropout

from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Conv2D, MaxPooling2D

In [ ]:
# vgg
def vgg1D(input_layer, conv_filters, conv_size, maxPool_size):
    vgg_layer = Conv1D(filters=conv_filters, kernel_size=conv_size, activation="relu")(input_layer)
    vgg_layer = BatchNormalization()(vgg_layer)
    vgg_layer = MaxPooling1D(pool_size=maxPool_size)(vgg_layer)
    return vgg_layer

def vgg2D(input_layer, conv_filters, conv_size, maxPool_size):
    vgg_layer = Conv2D(filters=conv_filters, kernel_size=conv_size, activation="relu")(input_layer)
    vgg_layer = BatchNormalization()(vgg_layer)
    vgg_layer = MaxPooling2D(pool_size=maxPool_size)(vgg_layer)
    return vgg_layer

In [ ]:
# model construction

# 1d cnn model for waveform 
def mcWave(xshape, yshape):
    input_layer = Input(xshape)
    hidden_layer = vgg1D(input_layer, 16, conv_size=128, maxPool_size=8)
    hidden_layer = vgg1D(hidden_layer, 8, conv_size=64, maxPool_size=8)
    hidden_layer = vgg1D(hidden_layer, 32, conv_size=256, maxPool_size=8)
    hidden_layer = Flatten()(hidden_layer)
    hidden_layer = Dense(32, activation="relu")(hidden_layer)
    hidden_layer = Dropout(0.2)(hidden_layer)
    output_layer = Dense(yshape)(hidden_layer)
    return Model(inputs=input_layer, outputs=output_layer)

# 2d cnn model for mel spectrogram input
def mcMel(xshape, yshape):
    input_layer = Input(xshape)
    hidden_layer = vgg2D(input_layer, 16, (8,8), (4,4))
    hidden_layer = vgg2D(hidden_layer, 8, (4,4), (2,2))
    hidden_layer = vgg2D(hidden_layer, 16, (4,4), (2,2))
    hidden_layer = Flatten()(hidden_layer)
    hidden_layer = Dense(32, activation="relu")(hidden_layer)
    hidden_layer = Dropout(0.2)(hidden_layer)
    output_layer = Dense(yshape)(hidden_layer)
    return Model(inputs=input_layer, outputs=output_layer)

# 2d cnn model for 64 mel filter banks
def mcMel64(xshape, yshape):
    input_layer = Input(xshape)
    hidden_layer = vgg2D(input_layer, 16, (8,8), (4,4))
    hidden_layer = vgg2D(hidden_layer, 8, (4,4), (2,2))
    hidden_layer = Flatten()(hidden_layer)
    hidden_layer = Dense(32, activation="relu")(hidden_layer)
    hidden_layer = Dropout(0.2)(hidden_layer)
    output_layer = Dense(yshape)(hidden_layer)
    return Model(inputs=input_layer, outputs=output_layer)

In [ ]:
# model summary
# mcMel64(XSHAPE_MEL64, YSHAPE).summary()
mcMel(XSHAPE_MEL128, YSHAPE).summary()
# mcMel(XSHAPE_MEL256, YSHAPE).summary()
# mcMel(XSHAPE_WAVE, YSHAPE).summary()

Training

In [ ]:
# training
def train(model, data_x, data_y, epochs, model_name, valid_x:list=None, valid_y:list=None, batch_size:int = 64): 
    valid_data = None
    if(not valid_x is None and not valid_y is None and len(valid_x) != 0 and len(valid_y) != 0):
        valid_data = (valid_x, valid_y)
    
    model.compile(optimizer="adam", loss="mse")
    model.fit(x=data_x, y=data_y, batch_size=batch_size, epochs=epochs, validation_data=valid_data)
    model.save(f"{MODEL_FOLDER_PATH}/{model_name}.h5")

    clear_session()
    gc.collect()

model 1: feature extraction

In [ ]:
# (020) model 1a - waveform, full parameter range, no y processing
# Since my device memory can not handle whole waveform dataset, only half of the dataset is used
model = mcWave(XSHAPE_WAVE, YSHAPE)

(train_data_x, train_data_y) = loadData(dataset="1",dataX_nm=DATAX_WAVE_HALF_NM,dataY_nm=DATAY_NO_PROC_HALF_NM)
(valid_data_x, valid_data_y) = loadData(dataset="2",dataX_nm=DATAX_WAVE_HALF_NM,dataY_nm=DATAY_NO_PROC_HALF_NM)

# 001 loss: 1193 - val_loss: 812
# 005 loss: 725.2937 - val_loss: 536.1645
# 010 loss: 603.7682 - val_loss: 510.9308
# 015 loss: 506.6414 - val_loss: 594.3563
# 020 loss: 426.2429 - val_loss: 509.3443 *
# 025 loss: 379.6118 - val_loss: 530.7617
# 030 loss: 336.2609 - val_loss: 547.8588
# 035 loss: 307.6765 - val_loss: 520.4119
# 040 loss: 286.2945 - val_loss: 521.8716
train(model, train_data_x, train_data_y, 1, "model1a", valid_x=valid_data_x, valid_y=valid_data_y, batch_size=4)

In [ ]:
# (150) model 1b - mel 64, full parameter range, no y processing
model = mcMel64(XSHAPE_MEL64, YSHAPE)

(train_data_x, train_data_y) = loadData(dataset="1",dataX_nm=DATAX_MEL64_NM,dataY_nm=DATAY_NO_PROC_NM)
(valid_data_x, valid_data_y) = loadData(dataset="2",dataX_nm=DATAX_MEL64_NM,dataY_nm=DATAY_NO_PROC_NM)

# 001 loss: 1301.3280 - val_loss: 995.3934
# 025 loss: 517.6314 - val_loss: 447.7486
# 050 loss: 462.3280 - val_loss: 436.9823
# 075 loss: 432.8425 - val_loss: 400.8932
# 100 loss: 415.4166 - val_loss: 397.9855
# 125 loss: 400.3361 - val_loss: 398.3643
# 150 loss: 392.6370 - val_loss: 392.0902*
# 175 loss: 382.2574 - val_loss: 398.9677
# 200 loss: 376.0954 - val_loss: 396.9700
# 225 loss: 366.0570 - val_loss: 403.7699
# 250 loss: 363.9658 - val_loss: 401.4442
train(model, train_data_x, train_data_y, 1, "model1b", valid_x=valid_data_x, valid_y=valid_data_y)

In [ ]:
# (100) model 1c - mel 128, full parameter range, no y processing
model = mcMel(XSHAPE_MEL128, YSHAPE)

(train_data_x, train_data_y) = loadData(dataset="1",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_NO_PROC_NM)
(valid_data_x, valid_data_y) = loadData(dataset="2",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_NO_PROC_NM)

# 001 loss: 1452.9558 - val_loss: 931.3066
# 050 loss: 429.6733 - val_loss: 384.3339
# 100 loss: 381.6813 - val_loss: 373.5758 *
# 150 loss: 362.4910 - val_loss: 387.3633
# 200 loss: 345.1278 - val_loss: 387.9102
# 250 loss: 335.1776 - val_loss: 384.5560
# 300 loss: 327.4211 - val_loss: 401.2144
train(model, train_data_x, train_data_y, 1, "model1c", valid_x=valid_data_x, valid_y=valid_data_y)

In [ ]:
# (080) model 1d - mel 256, full parameter range, no y processing
model = mcMel(XSHAPE_MEL256, YSHAPE)

(train_data_x, train_data_y) = loadData(dataset="1",dataX_nm=DATAX_MEL256_NM,dataY_nm=DATAY_NO_PROC_NM)
(valid_data_x, valid_data_y) = loadData(dataset="2",dataX_nm=DATAX_MEL256_NM,dataY_nm=DATAY_NO_PROC_NM)

# 001 loss: 985.4310 - val_loss: 626
# 010 loss: 541.8529 - val_loss: 433.1351
# 020 loss: 456.9853 - val_loss: 409.4021
# 030 loss: 407.5519 - val_loss: 391.5190
# 040 loss: 384.1865 - val_loss: 384.1649
# 050 loss: 360.6243 - val_loss: 386.2763
# 060 loss: 346.4266 - val_loss: 401.5240
# 070 loss: 331.0762 - val_loss: 384.8035 *
# 080 loss: 324.4471 - val_loss: 399.1465
# 090 loss: 313.3227 - val_loss: 397.2042
# 100 loss: 304.8206 - val_loss: 395.2017
# 110 loss: 298.0748 - val_loss: 406.3210
train(model, train_data_x, train_data_y, 1, "model1d", valid_x=valid_data_x, valid_y=valid_data_y, batch_size=16)

model 2: Processing Data Y

In [ ]:
# (250) model 2a - mel 128, vibrate scale up, full parameter range

model = mcMel(XSHAPE_MEL128, YSHAPE)

(train_data_x, train_data_y) = loadData(dataset="1",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_NM)
(valid_data_x, valid_data_y) = loadData(dataset="2",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_NM)

# 001 loss: 1487.4447 - val_loss: 1071.7545
# 025 loss: 529.8732 - val_loss: 430.2740
# 050 loss: 487.0937 - val_loss: 495.2896
# 075 loss: 461.2807 - val_loss: 455.2355
# 100 loss: 445.6952 - val_loss: 433.1125
# 125 loss: 437.2981 - val_loss: 458.0020
# 150 loss: 429.7370 - val_loss: 531.9053
# 175 loss: 420.7566 - val_loss: 560.1247
# 200 loss: 412.7821 - val_loss: 431.0232
# 225 loss: 405.2132 - val_loss: 483.9254
# 250 loss: 400.6638 - val_loss: 412.3844 *
# 275 loss: 395.7731 - val_loss: 469.0576
# 300 loss: 391.9837 - val_loss: 478.0786
# 325 loss: 386.8188 - val_loss: 458.9661
# 350 loss: 383.2034 - val_loss: 430.2350
# 375 loss: 379.2342 - val_loss: 501.6038
train(model, train_data_x, train_data_y, 1, "model2a", valid_x=valid_data_x, valid_y=valid_data_y)

In [ ]:
# (425) model 2b1 - mel 128, vibrate scale up, processed gen, full parameter range

model = mcMel(XSHAPE_MEL128, YSHAPE)
# model = load_model(f"{MODELFOLDER_PATH}/model6a.h5")

(train_data_x, train_data_y) = loadData(dataset="1",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_GEN_NM)
(valid_data_x, valid_data_y) = loadData(dataset="2",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_GEN_NM)

# 001 loss: 1812.8190 - val_loss: 1281.5427
# 025 loss: 589.2810 - val_loss: 446.8871
# 050 loss: 529.7285 - val_loss: 434.5656
# 075 loss: 494.6658 - val_loss: 417.4658
# 100 loss: 478.5611 - val_loss: 405.2747
# 125 loss: 462.3292 - val_loss: 432.9203
# 150 loss: 446.8837 - val_loss: 409.8909
# 175 loss: 437.1096 - val_loss: 409.1732
# 200 loss: 428.1109 - val_loss: 400.7753
# 225 loss: 420.6072 - val_loss: 423.8295 * 
# 250 loss: 412.8520 - val_loss: 461.6810
# 275 loss: 409.1534 - val_loss: 441.8086
# 300 loss: 403.5663 - val_loss: 416.8514 *
# 325 loss: 398.6558 - val_loss: 422.8240
# 350 loss: 397.0928 - val_loss: 425.2311
# 375 loss: 391.1743 - val_loss: 436.6808
# 400 loss: 386.9486 - val_loss: 434.1211
# 425 loss: 383.4555 - val_loss: 409.2722 *
# 450 loss: 381.4274 - val_loss: 426.1567
# 475 loss: 379.0038 - val_loss: 420.6597
# 500 loss: 377.0009 - val_loss: 440.5930
# 525 loss: 373.1816 - val_loss: 420.7093
# 550 loss: 370.2161 - val_loss: 421.0287
train(model, train_data_x, train_data_y, 1, "model2b1", valid_x=valid_data_x, valid_y=valid_data_y)

In [ ]:
# (300) model 2b2 - mel 128, vibrate scale up, processed gen, full parameter range

model = mcMel(XSHAPE_MEL128, YSHAPE)
# model = load_model(f"{MODELFOLDER_PATH}/model6a.h5")

(train_data_x, train_data_y) = loadData(dataset="1",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_GEN_NM)
(valid_data_x, valid_data_y) = loadData(dataset="2",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_GEN_NM)

# 001 loss: loss: 1563.6788 - val_loss: 1170.6083
# 025 loss: 553.0649 - val_loss: 528.1212
# 050 loss: 505.1127 - val_loss: 471.5759
# 075 loss: 485.3903 - val_loss: 417.1454
# 100 loss: 469.0105 - val_loss: 414.3436
# 125 loss: 458.3802 - val_loss: 409.2711
# 150 loss: 440.6630 - val_loss: 406.1250
# 175 loss: 432.3932 - val_loss: 396.1932
# 200 loss: 423.1768 - val_loss: 419.0807
# 225 loss: 420.6703 - val_loss: 393.5967
# 250 loss: 412.2075 - val_loss: 400.1005
# 275 loss: 407.7595 - val_loss: 403.5808
# 300 loss: 401.7838 - val_loss: 403.4929 * 
# 325 loss: 398.6189 - val_loss: 407.5320
# 350 loss: 391.8011 - val_loss: 410.4254
# 375 loss: 387.1917 - val_loss: 406.5925
# 400 loss: 385.5360 - val_loss: 416.1210
train(model, train_data_x, train_data_y, 1, "model2b2", valid_x=valid_data_x, valid_y=valid_data_y)

In [ ]:
# (300) model 2b3* - mel 128, vibrate scale up, processed gen, full parameter range

model = mcMel(XSHAPE_MEL128, YSHAPE)
# model = load_model(f"{MODELFOLDER_PATH}/model6a.h5")

(train_data_x, train_data_y) = loadData(dataset="1",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_GEN_NM)
(valid_data_x, valid_data_y) = loadData(dataset="2",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_GEN_NM)

# 001 loss: 1791.0804 - val_loss: 1215.7341
# 025 loss: 556.4993 - val_loss: 447.8711
# 050 loss: 512.0079 - val_loss: 495.1118
# 075 loss: 487.8735 - val_loss: 441.3969
# 100 loss: 470.4938 - val_loss: 416.6122
# 125 loss: 457.2826 - val_loss: 427.6158
# 150 loss: 448.6826 - val_loss: 454.4707 *
# 175 loss: 438.6080 - val_loss: 396.3398 *
# 200 loss: 431.9781 - val_loss: 415.8568
# 225 loss: 419.5443 - val_loss: 399.9714
# 250 loss: 418.4087 - val_loss: 397.4957 *
# 275 loss: 411.7956 - val_loss: 414.0408
# 300 loss: 406.1541 - val_loss: 394.8354 * 
# 325 loss: 399.6231 - val_loss: 400.0864
# 350 loss: 398.1526 - val_loss: 439.3050
# 375 loss: 391.7644 - val_loss: 405.9939
# 400 loss: 387.9780 - val_loss: 466.5602
train(model, train_data_x, train_data_y, 1, "model2b3", valid_x=valid_data_x, valid_y=valid_data_y)

In [ ]:
# (325) model 2c1 - mel 128, vibrate scale up, processed gen + bre, full parameter range

model = mcMel(XSHAPE_MEL128, YSHAPE)

(train_data_x, train_data_y) = loadData(dataset="1",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_GEN_BRE_NM)
(valid_data_x, valid_data_y) = loadData(dataset="2",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_GEN_BRE_NM)

# 001 loss: 1611.9681 - val_loss: 1131.6368
# 025 loss: 563.9680 - val_loss: 449.4275
# 050 loss: 516.9772 - val_loss: 444.9075
# 075 loss: 493.2162 - val_loss: 412.5422
# 100 loss: 477.1926 - val_loss: 416.3568
# 125 loss: 463.6755 - val_loss: 413.1193
# 150 loss: 452.2112 - val_loss: 411.5794
# 175 loss: 444.7369 - val_loss: 444.6857
# 200 loss: 435.8983 - val_loss: 421.6447
# 225 loss: 425.5401 - val_loss: 420.4705
# 250 loss: 419.7874 - val_loss: 415.6989
# 275 loss: 413.5142 - val_loss: 418.5410
# 300 loss: 407.7443 - val_loss: 413.4102 *
# 325 loss: 403.5949 - val_loss: 412.0166 *
# 350 loss: 400.1649 - val_loss: 413.7079
# 375 loss: 397.5446 - val_loss: 419.2021
# 400 loss: 394.1733 - val_loss: 418.2340
# 425 loss: 391.0244 - val_loss: 425.3868
train(model, train_data_x, train_data_y, 1, "model2c1", valid_x=valid_data_x, valid_y=valid_data_y)

In [ ]:
# (450) model 2c2 - mel 128, vibrate scale up, processed gen + bre, full parameter range

model = mcMel(XSHAPE_MEL128, YSHAPE)

(train_data_x, train_data_y) = loadData(dataset="1",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_GEN_BRE_NM)
(valid_data_x, valid_data_y) = loadData(dataset="2",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_GEN_BRE_NM)

# 001 loss: 1682.7423 - val_loss: 1202.4968
# 025 loss: 556.7562 - val_loss: 479.3653
# 050 loss: 507.2174 - val_loss: 581.8183
# 075 loss: 480.5387 - val_loss: 407.7279
# 100 loss: 468.3243 - val_loss: 404.8054
# 125 loss: 452.3046 - val_loss: 448.5900
# 150 loss: 441.9933 - val_loss: 416.8430
# 175 loss: 432.4933 - val_loss: 426.4548
# 200 loss: 427.2293 - val_loss: 410.2086
# 225 loss: 423.0116 - val_loss: 425.3227 * 
# 250 loss: 417.0458 - val_loss: 425.3330 *
# 275 loss: 410.5042 - val_loss: 461.7872
# 300 loss: 404.8000 - val_loss: 419.4872 *
# 325 loss: 401.3165 - val_loss: 415.4628 *
# 350 loss: 398.4255 - val_loss: 415.5319 * 
# 375 loss: 394.4145 - val_loss: 435.9538
# 400 loss: 394.6408 - val_loss: 465.1625
# 425 loss: 388.5503 - val_loss: 416.2935
# 450 loss: 383.1222 - val_loss: 414.2978 *
# 475 loss: 381.7888 - val_loss: 430.1852
# 500 loss: 381.3143 - val_loss: 448.2125
# 525 loss: 378 - val_loss: 417
# 550 loss: 375.2482 - val_loss: 446.6959

train(model, train_data_x, train_data_y, 1, "model2c2", valid_x=valid_data_x, valid_y=valid_data_y)

In [ ]:
# (300) model 2c3* - mel 128, vibrate scale up, processed gen + bre, full parameter range

model = mcMel(XSHAPE_MEL128, YSHAPE)

(train_data_x, train_data_y) = loadData(dataset="1",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_GEN_BRE_NM)
(valid_data_x, valid_data_y) = loadData(dataset="2",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_GEN_BRE_NM)

# 001 loss: 1668.2982 - val_loss: 1117.8893
# 025 loss: 572.9258 - val_loss: 482.2334
# 050 loss: 514.9022 - val_loss: 445.1064
# 075 loss: 491.5103 - val_loss: 434.9570
# 100 loss: 472.6076 - val_loss: 408.8380
# 125 loss: 458.4455 - val_loss: 427.5649
# 150 loss: 448.1552 - val_loss: 415.5061
# 175 loss: 440.4944 - val_loss: 418.8634
# 200 loss: 434.1789 - val_loss: 413.1811
# 225 loss: 425.6351 - val_loss: 421.3347
# 250 loss: 414.3134 - val_loss: 434.8697 *
# 275 loss: 407.8704 - val_loss: 410.8754 *
# 300 loss: 403.5162 - val_loss: 407.7198 *
# 325 loss: 397.6183 - val_loss: 449.6312
# 350 loss: 394.8601 - val_loss: 415.8656
# 375 loss: 392.9868 - val_loss: 420.0578
# 400 loss: 387.5048 - val_loss: 423.3068
train(model, train_data_x, train_data_y, 1, "model2c3", valid_x=valid_data_x, valid_y=valid_data_y)

In [ ]:
# (275) model 2d1 - mel 128, vibrate scale up, processed gen + bri, full parameter range

model = mcMel(XSHAPE_MEL128, YSHAPE)

(train_data_x, train_data_y) = loadData(dataset="1",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_GEN_BRI_NM)
(valid_data_x, valid_data_y) = loadData(dataset="2",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_GEN_BRI_NM)

# 001 loss: 1594.3430 - val_loss: 1166.0999
# 025 loss: 576.8270 - val_loss: 473.2868
# 050 loss: 524.5601 - val_loss: 473.2828
# 075 loss: 502.4763 - val_loss: 410.7623
# 100 loss: 481.9157 - val_loss: 443.3833
# 125 loss: 468.7600 - val_loss: 408.9103
# 150 loss: 461.8169 - val_loss: 415.3096
# 175 loss: 446.0678 - val_loss: 417.6773
# 200 loss: 439.2602 - val_loss: 404.4516
# 225 loss: 430.6883 - val_loss: 427.6917
# 250 loss: 423.9079 - val_loss: 427.3008 *
# 275 loss: 413.9656 - val_loss: 404.8669 *
# 300 loss: 407.9448 - val_loss: 432.9791
# 325 loss: 405.4808 - val_loss: 430.1374
# 350 loss: 398.4392 - val_loss: 415.2329
# 375 loss: 397.3951 - val_loss: 409.5854
# 400 loss: 392.6377 - val_loss: 415.2363
train(model, train_data_x, train_data_y, 1, "model2d1", valid_x=valid_data_x, valid_y=valid_data_y)

In [ ]:
# (275) model 2d2* - mel 128, vibrate scale up, processed gen + bri, full parameter range

model = mcMel(XSHAPE_MEL128, YSHAPE)

(train_data_x, train_data_y) = loadData(dataset="1",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_GEN_BRI_NM)
(valid_data_x, valid_data_y) = loadData(dataset="2",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_GEN_BRI_NM)

# 001 loss: 1661.3502 - val_loss: 1074.6798
# 025 loss: 556.0868 - val_loss: 451.7941
# 050 loss: 513.6392 - val_loss: 437.6575
# 075 loss: 489.3364 - val_loss: 423.8499
# 100 loss: 473.0967 - val_loss: 473.0736
# 125 loss: 457.8735 - val_loss: 405.1814
# 150 loss: 448.6401 - val_loss: 401.2310
# 175 loss: 440.0706 - val_loss: 408.8427
# 200 loss: 430.7902 - val_loss: 414.9513
# 225 loss: 424.3821 - val_loss: 420.6793
# 250 loss: 420.2105 - val_loss: 423.9818 *
# 275 loss: 414.5349 - val_loss: 401.7042 *
# 300 loss: 409.4262 - val_loss: 428.6040
# 325 loss: 402.0507 - val_loss: 416.8754
# 350 loss: 397.1090 - val_loss: 408.3976
# 375 loss: 398.4076 - val_loss: 406.1562
# 400 loss: 390.1893 - val_loss: 414.8922
train(model, train_data_x, train_data_y, 1, "model2d2", valid_x=valid_data_x, valid_y=valid_data_y)

In [ ]:
# (275) model 2d3 - mel 128, vibrate scale up, processed gen + bri, full parameter range

model = mcMel(XSHAPE_MEL128, YSHAPE)

(train_data_x, train_data_y) = loadData(dataset="1",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_GEN_BRI_NM)
(valid_data_x, valid_data_y) = loadData(dataset="2",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_GEN_BRI_NM)

# 001 loss: 1571.2820 - val_loss: 1120.9987
# 025 loss: 570.3436 - val_loss: 463.5702
# 050 loss: 521.8278 - val_loss: 427.0589
# 075 loss: 496.7997 - val_loss: 412.7359
# 100 loss: 472.6187 - val_loss: 422.8802
# 125 loss: 457.6522 - val_loss: 400.8858
# 150 loss: 449.0911 - val_loss: 441.5378
# 175 loss: 440.6806 - val_loss: 413.7066
# 200 loss: 430.9173 - val_loss: 407.8697
# 225 loss: 425.8330 - val_loss: 425.0571 *
# 250 loss: 417.3460 - val_loss: 425.6826 *
# 275 loss: 413.2396 - val_loss: 408.0042 *
# 300 loss: 410.7764 - val_loss: 410.8354
# 325 loss: 403.2354 - val_loss: 411.7202
# 350 loss: 399.1380 - val_loss: 409.4993
# 375 loss: 397.3383 - val_loss: 418.8696
train(model, train_data_x, train_data_y, 1, "model2d3", valid_x=valid_data_x, valid_y=valid_data_y)

In [ ]:
# (275) model 2e1 - mel 128, vibrate scale up, processed gen + gwl, full parameter range

model = mcMel(XSHAPE_MEL128, YSHAPE)

(train_data_x, train_data_y) = loadData(dataset="1",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_GEN_GWL_NM)
(valid_data_x, valid_data_y) = loadData(dataset="2",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_GEN_GWL_NM)

# 001 loss: 1718.1310 - val_loss: 1425.3628
# 025 loss: 576.3031 - val_loss: 450.9186
# 050 loss: 513.4543 - val_loss: 414.5903
# 075 loss: 486.7133 - val_loss: 423.5378
# 100 loss: 465.9051 - val_loss: 410.4929
# 125 loss: 454.1998 - val_loss: 427.8911
# 150 loss: 443.8425 - val_loss: 408.1984
# 175 loss: 435.0164 - val_loss: 402.5806
# 200 loss: 428.8726 - val_loss: 397.9141
# 225 loss: 420.9866 - val_loss: 427.7891 *
# 250 loss: 416.9972 - val_loss: 415.8062 *
# 275 loss: 410.3405 - val_loss: 404.5659 *
# 300 loss: 404.2950 - val_loss: 507.6013
# 325 loss: 401.6892 - val_loss: 410.6785
# 350 loss: 397.2623 - val_loss: 420.6295
# 375 loss: 393.9316 - val_loss: 416.0857
train(model, train_data_x, train_data_y, 1, "model2e1", valid_x=valid_data_x, valid_y=valid_data_y)

In [ ]:
# (475) model 2e2 - mel 128, vibrate scale up, processed gen + gwl, full parameter range

model = mcMel(XSHAPE_MEL128, YSHAPE)

(train_data_x, train_data_y) = loadData(dataset="1",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_GEN_GWL_NM)
(valid_data_x, valid_data_y) = loadData(dataset="2",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_GEN_GWL_NM)

# 001 loss: 1718.1310 - val_loss: 1425.3628
# 025 loss: 562.2855 - val_loss: 462.2986
# 050 loss: 515.3514 - val_loss: 493.1979
# 075 loss: 487.1183 - val_loss: 411.0853
# 100 loss: 471.6380 - val_loss: 418.1647
# 125 loss: 462.3184 - val_loss: 439.6248
# 150 loss: 447.7553 - val_loss: 414.6300
# 175 loss: 440.8737 - val_loss: 452.1861 *
# 200 loss: 432.3351 - val_loss: 432.9413 *
# 225 loss: 427.3504 - val_loss: 419.1671 *
# 250 loss: 419.8763 - val_loss: 407.4744 *
# 275 loss: 416.3602 - val_loss: 416.9554 *
# 300 loss: 410.7128 - val_loss: 419.5520
# 325 loss: 403.2137 - val_loss: 411.4487 *
# 350 loss: 399.3268 - val_loss: 428.2825
# 375 loss: 397.3120 - val_loss: 424.2336
# 400 loss: 392.0657 - val_loss: 411.1647 *
# 425 loss: 390.2519 - val_loss: 411.8023 *
# 450 loss: 386.2309 - val_loss: 414.8119
# 475 loss: 386.0637 - val_loss: 411.5648 *
# 500 loss: 379.5587 - val_loss: 415.7087
# 525 loss: 380.4120 - val_loss: 414.2491
# 550 loss: 377.8878 - val_loss: 413.5557
# 575 loss: 379.7215 - val_loss: 412.3418
train(model, train_data_x, train_data_y, 1, "model2e2", valid_x=valid_data_x, valid_y=valid_data_y)

In [ ]:
# (300) model 2e3 - mel 128, vibrate scale up, processed gen + gwl, full parameter range

model = mcMel(XSHAPE_MEL128, YSHAPE)

(train_data_x, train_data_y) = loadData(dataset="1",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_GEN_GWL_NM)
(valid_data_x, valid_data_y) = loadData(dataset="2",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_VIB_GEN_GWL_NM)

# 001 loss: 1585.5270 - val_loss: 1193.4951
# 025 loss: 555.9403 - val_loss: 435.8211
# 050 loss: 509.0179 - val_loss: 421.4337
# 075 loss: 486.2159 - val_loss: 435.4472
# 100 loss: 471.5846 - val_loss: 408.4354
# 125 loss: 459.0171 - val_loss: 420.6580
# 150 loss: 450.4613 - val_loss: 429.7076
# 175 loss: 442.7292 - val_loss: 444.0904
# 200 loss: 432.4766 - val_loss: 411.6470
# 225 loss: 426.9782 - val_loss: 419.0509
# 250 loss: 419.1260 - val_loss: 407.1523
# 275 loss: 414.0470 - val_loss: 404.7534
# 300 loss: 409.2673 - val_loss: 413.9771 *
# 325 loss: 406.0016 - val_loss: 414.4080
# 350 loss: 400.5526 - val_loss: 450.9892
# 376 loss: 403 - val_loss: 439
# 400 loss: 397.4006 - val_loss: 412.4081
# 425 loss: 393.8844 - val_loss: 419.6578
# 450 loss: 390.8320 - val_loss: 428.4792
# 475 loss: 386.4795 - val_loss: 424.0874
# 500 loss: 384.5521 - val_loss: 441.3961
train(model, train_data_x, train_data_y, 1, "model2e3", valid_x=valid_data_x, valid_y=valid_data_y)

Model 3: limit parameter range

In [ ]:
# (200) model 3_1* - mel 128, processed gen, limited parameter range, min max noramlization on y is applied

model = mcMel(XSHAPE_MEL128, YSHAPE)

(train_data_x, train_data_y) = loadData(dataset="4",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_GEN_MINMAXNORM)
(valid_data_x, valid_data_y) = loadData(dataset="5",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_GEN_MINMAXNORM)

# 001 loss: 1477.0626 - val_loss: 1169.6171
# 025 loss: 568.0341 - val_loss: 527.7632
# 050 loss: 525.1954 - val_loss: 505.3529
# 075 loss: 500.2196 - val_loss: 511.9316*
# 100 loss: 481.5819 - val_loss: 538.9978
# 125 loss: 467.5703 - val_loss: 510.7985
# 150 loss: 453.5608 - val_loss: 529.3759
# 175 loss: 446.3099 - val_loss: 513.3421
# 200 loss: 436.2967 - val_loss: 495.7828*
# 225 loss: 422.9414 - val_loss: 505.6229
# 250 loss: 409.5604 - val_loss: 503.9065
# 275 loss: 400.7654 - val_loss: 515.0219
# 300 loss: 397.2670 - val_loss: 514.0422
train(model, train_data_x, train_data_y, 1, "model3_1", valid_x=valid_data_x, valid_y=valid_data_y)

In [ ]:
# (100) model 3_2 - mel 128, processed gen, limited parameter range, min max noramlization on y is applied

model = mcMel(XSHAPE_MEL128, YSHAPE)

(train_data_x, train_data_y) = loadData(dataset="4",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_GEN_MINMAXNORM)
(valid_data_x, valid_data_y) = loadData(dataset="5",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_GEN_MINMAXNORM)

# 001 loss: 1477.0626 - val_loss: 1169.6171
# 025 loss: 537.6884 - val_loss: 537.5815*
# 050 loss: 493.0067 - val_loss: 502.9045*
# 075 loss: 468.9686 - val_loss: 500.7999*
# 100 loss: 447.6135 - val_loss: 493.2443*
# 125 loss: 436.8156 - val_loss: 519.4130
# 150 loss: 425.8451 - val_loss: 509.1278
# 175 loss: 421.0120 - val_loss: 506.1464
# 200 loss: 411.1922 - val_loss: 518.1401
train(model, train_data_x, train_data_y, 1, "model3_2", valid_x=valid_data_x, valid_y=valid_data_y)

In [ ]:
# (150) model 3_3 - mel 128, processed gen, limited parameter range, min max noramlization on y is applied

model = mcMel(XSHAPE_MEL128, YSHAPE)

(train_data_x, train_data_y) = loadData(dataset="4",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_GEN_MINMAXNORM)
(valid_data_x, valid_data_y) = loadData(dataset="5",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_GEN_MINMAXNORM)

# 001 loss: 1385.2886 - val_loss: 1118.0209
# 025 loss: 559.5919 - val_loss: 564.5846*
# 050 loss: 517.2621 - val_loss: 548.2025*
# 075 loss: 489.6852 - val_loss: 532.1967*
# 100 loss: 473.2576 - val_loss: 507.6737*
# 125 loss: 460.3712 - val_loss: 525.2838
# 150 loss: 451.6604 - val_loss: 504.6861*
# 175 loss: 434.0170 - val_loss: 509.2962
# 200 loss: 424.3714 - val_loss: 544.9255
# 225 loss: 420.3195 - val_loss: 536.6537
# 250 loss: 412.3930 - val_loss: 522.2183
train(model, train_data_x, train_data_y, 1, "model3_3", valid_x=valid_data_x, valid_y=valid_data_y)

For controlling the training progress

In [ ]:
train(model, train_data_x, train_data_y, 25, "model3_3", valid_x=valid_data_x, valid_y=valid_data_y)

In [5]:
model = load_model(f"{MODEL_FOLDER_PATH}/model3_3.h5")
(train_data_x, train_data_y) = loadData(dataset="4",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_GEN_MINMAXNORM)
(valid_data_x, valid_data_y) = loadData(dataset="5",dataX_nm=DATAX_MEL128_NM,dataY_nm=DATAY_PROC_GEN_MINMAXNORM)